In [1]:
import os
import glob

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.ticker as ticker
import IPython.display
import pygame
from pygame import gfxdraw
import cv2

pygame.init()

def video_write(_path, _outpath, fps = 30):
    paths = glob.glob(os.path.join(_path, r"[0-9]*.png"))
    paths.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))

    height, width, layers = cv2.imread(paths[0]).shape
    size = (width, height)

    out = cv2.VideoWriter(_outpath, cv2.VideoWriter_fourcc(*"DIVX"), fps, size)
    for p in paths:
        out.write(cv2.imread(p))
    out.release()

def draw_circle(surface, pos, radius, color):
    gfxdraw.aacircle(surface, pos[0], pos[1], radius, color)
    gfxdraw.filled_circle(surface, pos[0], pos[1], radius, color)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# 진자

진자는 진동자의 준말이다.

진자의 추가 결국 그 당시의 각과 중력에 따라 달라진다.

따라서 당시의 각가속도는 다음과 같다.

$${\frac{d^2\theta}{dt^2}} = \frac{-g \sin\theta}{l}$$

In [11]:
class Pendulum:
    g = +30
    def __init__(self, pivot, m, r, init_theta = 1):
        self.pivot = pivot
        self.mass = m
        self.radius = r
        self.angle = init_theta
        
        self.accel = np.zeros(2)
        self.velocity = np.zeros(2)
        self.pos = self.pivot + self.radius * np.array((np.sin(init_theta), +np.cos(init_theta)))
        
        self.angle_velocity = 0
        self.angle_accel = 0
        
    def update(self):
        self.angle_accel = - self.g * np.sin(self.angle) / self.radius
        self.angle_velocity += self.angle_accel
        self.angle += self.angle_velocity
        self.pos = self.pivot + self.radius * np.array((np.sin(self.angle), +np.cos(self.angle)))

    def draw(self, surface):
        pygame.draw.circle(surface, (0, 0, 0), self.pivot.astype(int), 2, )
        pygame.draw.line(surface, (0, 0, 0), self.pivot.astype(int), self.pos.astype(int), 1)
        pygame.draw.circle(surface, (0, 0, 0), self.pos.astype(int), 10, 0)
        
        

In [12]:
width, height = 400, 400


surface = pygame.Surface((width, height), flags = pygame.DOUBLEBUF | pygame.HWSURFACE)
# surface: pygame.Surface = pygame.display.set_mode((width, height), pygame.DOUBLEBUF | pygame.HWSURFACE)


_name = "pendulum-example"
_path = f"img/{_name}/"
_outpath = f"out/{_name}.mp4"

if not os.path.isdir(_path):
    os.mkdir(_path)

pendulum = Pendulum(np.array((width/2, 10)), 5, 100, 0.5)
pendulum2 = Pendulum(pendulum.pos, 5, 100, 1)
pendulum3 = Pendulum(pendulum.pos, 5, 100, -0.5)
for frame in range(200):
    surface.fill((255, 255, 255))

    pendulum.update()
    pendulum2.pivot = pendulum.pos
    pendulum3.pivot = pendulum.pos
    pendulum2.update()
    pendulum3.update()

    pendulum.draw(surface)
    pendulum2.draw(surface)
    pendulum3.draw(surface)

    if True:
        pygame.image.save(surface, _path + f"{frame}.png")

video_write(_path, _outpath)